<h1><b>Identification of important events and there words from the document

<h3><b>Import requried libraries

In [1]:
import pandas as pd
import string
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.frequent_patterns import apriori,association_rules
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import preprocessor as p
from scipy.stats import entropy

nlp = spacy.load('en_core_web_sm')
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

<h3><b>Load twitter dataset from kaggle

In [111]:
df = pd.read_csv('datasets_corona_May_Tweets.csv',engine='python')

In [112]:
df=df[['created_at','text']]

In [113]:
#Identify Unique hashtags
utags = []
def getTags(x):
    tag = " ".join([str(i)  for i in x.split() if i.startswith("#") ])
    for i in x.split():
        if i.startswith("#"):
            if i not in utags:
                utags.append(i)
    return(tag)

In [114]:
df['hashTag'] = df['text'].apply(lambda x:getTags(x))
df['date'] = pd.to_datetime(df['created_at'])

In [7]:
no_tweets = len(df)
no_tags = len(utags)
staring_dateTime = '2020-05-02 00:59'
ending_dateTime = '2020-05-02 00:03'

In [8]:
clean_data =df['text'].apply(lambda x: p.clean(x))

In [120]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    #after tweepy preprocessing the colon symbol left remain after   
    tweet = re.sub(r':', '', text_rc)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #filter using NLTK library append it to a string
    tokens = re.split('\W+', tweet)    # tokenization
    text = [ps.stem(word) for word in tokens ]#if word not in stopword]  # remove stopwords and stemming
    return text

def clean_textWithoutSteam(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    #after tweepy preprocessing the colon symbol left remain after    
    tweet = re.sub(r':', '', text_rc)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #filter using NLTK library append it to a string
    tokens = re.split('\W+', tweet)    # tokenization
    text = [word for word in tokens ]#if word not in stopword]  # remove stopwords
    return text

<h3><b>Create Dataframe of DTM of with stemming and without stemming

In [13]:
ex = []
ex.append(clean_data[2])
ex.append(clean_data[7])
ex.append(clean_data[27])
example = pd.DataFrame(ex)

In [14]:
def countVect(text):
    countVectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b",analyzer=clean_textWithoutSteam,stop_words=None) 
    countVector = countVectorizer.fit_transform(text)
    dtm = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
    
    countVectorizer1 = CountVectorizer(token_pattern=r"(?u)\b\w+\b",analyzer=clean_text,stop_words=None) 
    countVector1 = countVectorizer1.fit_transform(text)
    dtm1 = pd.DataFrame(countVector1.toarray(), columns=countVectorizer1.get_feature_names())
    
    return (dtm,dtm1)

df_withoutStem, df_withStem = countVect(clean_data)

In [15]:
#example data frame
df_exWithoutStem, df_exWithStem = countVect(example[0])

In [16]:
no_wordsTweets = df_withoutStem.sum(axis=1).sum()//3116
no_wordsDataset = len(df_withoutStem.columns)

In [17]:
info = pd.DataFrame([no_tweets,no_tags,no_wordsTweets,no_wordsDataset,staring_dateTime,ending_dateTime],index=['Number of Tweets',
                                                                                                               'Number of Unique Hashtags',
                                                                                                               'Average words per tweets',
                                                                                                               'Total number of words in dataset',
                                                                                                               'Starting dateTime','Ending dateTime'],
                   columns=['Dataset Details'])

In [19]:
#Dataset detailed information
info

,Dataset Details
Number of Tweets,3116
Number of Unique Hashtags,939
Average words per tweets,10
Total number of words in dataset,4729
Starting dateTime,2020-05-02 00:59
Ending dateTime,2020-05-02 00:03


<h3><b> Get Association of stemmed tweets and without stemmed tweets

In [20]:
def getAssociation(dtm1,dtm):
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    word_sets_withStem = dtm1.iloc[:,1:].applymap(encode_units)
    word_sets_withoutStem = dtm.iloc[:,1:].applymap(encode_units)
    
    prob_words_withStem = apriori(word_sets_withStem,min_support=0.01,verbose=1,use_colnames=True)
    prob_words_withoutStem = apriori(word_sets_withoutStem,min_support=0.01,verbose=1,use_colnames=True)
    
    rules_withStem = association_rules(prob_words_withStem,min_threshold=0)
    rules_withoutStem = association_rules(prob_words_withoutStem,min_threshold=0)
    
    return(rules_withStem,rules_withoutStem)

In [21]:
rules_withStem, rules_withoutStem = getAssociation(df_withStem,df_withoutStem)

Processing 48 combinations | Sampling itemset size 12116
Processing 48 combinations | Sampling itemset size 12116


In [22]:
#example data
rules_exwithStem, rules_exwithoutStem = getAssociation(df_exWithStem,df_exWithoutStem)

Processing 18 combinations | Sampling itemset size 65
Processing 14 combinations | Sampling itemset size 75


In [23]:
#example data
asso_exwithStem = rules_exwithStem[['antecedents','consequents','support','confidence']]
asso_exwithoutStem = rules_exwithoutStem[['antecedents','consequents','support','confidence']]

asso_exwithStem.columns = pd.MultiIndex.from_product([['With_Stemming'], asso_exwithStem.columns])
asso_exwithoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], asso_exwithoutStem.columns])

result_ex = pd.concat([asso_exwithStem, asso_exwithoutStem], axis = 1)

In [24]:
#rules_withoutStem.head()

In [25]:
asso_withStem = rules_withStem[['antecedents','consequents','support','confidence']]
asso_withoutStem = rules_withoutStem[['antecedents','consequents','support','confidence']]

asso_withStem.columns = pd.MultiIndex.from_product([['With_Stemming'], asso_withStem.columns])
asso_withoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], asso_withoutStem.columns])

result = pd.concat([asso_withStem, asso_withoutStem], axis = 1)

In [26]:
result.head()
#support = Probability
#confidence = Conditinal Probability

With_Stemming                                    Without_Stemming  \
    antecedents   consequents   support confidence      antecedents   
0      (consid)         (ago)  0.049743   0.981013       (consider)   
1         (ago)      (consid)  0.049743   0.933735            (ago)   
2  (conspiraci)         (ago)  0.049743   0.987261     (conspiracy)   
3         (ago)  (conspiraci)  0.049743   0.933735            (ago)   
4        (fact)         (ago)  0.049743   0.928144          (facts)   

                                      
    consequents   support confidence  
0         (ago)  0.049743   0.993590  
1    (consider)  0.049743   0.933735  
2         (ago)  0.049743   0.987261  
3  (conspiracy)  0.049743   0.933735  
4         (ago)  0.049743   0.981013

In [27]:
def getRanks(x,y):
    rank = x.sort_values('confidence',axis = 0, ascending = False, inplace = False)[['antecedents','consequents','confidence']]
    rank.reset_index(drop=True,inplace=True)
    rank['rank'] = rank.index.values
    col = rank.columns.tolist()
    col = col[-1:] + col[:-1]
    
    rank1 = y.sort_values('confidence',axis = 0, ascending = False, inplace = False)[['antecedents','consequents','confidence']]
    rank1.reset_index(drop=True,inplace=True)
    rank1['rank'] = rank1.index.values
    
    return(rank[col],rank1[col])

In [28]:
rank_asso_withStem, rank_asso_withoutStem = getRanks(rules_withStem, rules_withoutStem)

In [29]:
#examle data
rank_asso_exwithStem, rank_asso_exwithoutStem = getRanks(rules_exwithStem, rules_exwithoutStem)
rank_asso_exwithStem.columns = pd.MultiIndex.from_product([['With_Stemming'], rank_asso_exwithStem.columns])
rank_asso_exwithoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], rank_asso_exwithoutStem.columns])

rank_asso_exresult = pd.concat([rank_asso_exwithStem, rank_asso_exwithoutStem], axis = 1)

In [30]:
#example df
rank_asso_exresult.head(5)

With_Stemming                                                    \
           rank            antecedents     consequents confidence   
0           0.0            (bioweapon)        (govern)        1.0   
1           1.0               (govern)     (bioweapon)        1.0   
2           2.0  (relat, report, toll)           (new)        1.0   
3           3.0    (report, new, toll)         (relat)        1.0   
4           4.0           (relat, new)  (report, toll)        1.0   

  Without_Stemming                                            \
              rank                               antecedents   
0                0                             (governments)   
1                1  (topsecret, governments, us, bioweapons)   
2                2              (topsecret, governments, us)   
3                3             (governments, us, bioweapons)   
4                4      (topsecret, governments, bioweapons)   

                                    
            consequents confidence  
0          (bioweapons)        1.0  
1              (inside)        1.0  
2  (inside, bioweapons)        1.0  
3   (topsecret, inside)        1.0  
4          (us, inside)        1.0

In [31]:
rank_asso_withStem.columns = pd.MultiIndex.from_product([['With_Stemming'], rank_asso_withStem.columns])
rank_asso_withoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], rank_asso_withoutStem.columns])

rank_asso_result = pd.concat([rank_asso_withStem, rank_asso_withoutStem], axis = 1)

In [32]:
rank_asso_result.head()

With_Stemming                           \
           rank              antecedents   
0             0            (tho, doctor)   
1             1         (te, never, ago)   
2             2     (theori, month, ago)   
3             3  (verifi, theori, month)   
4             4   (never, theori, month)   

                                                    Without_Stemming  \
                             consequents confidence             rank   
0  (help, nursesnow, shut, claim, front)        1.0              0.0   
1   (month, thing, theori, fact, verifi)        1.0              1.0   
2       (te, thing, fact, never, verifi)        1.0              2.0   
3          (te, thing, fact, never, ago)        1.0              3.0   
4         (te, thing, fact, verifi, ago)        1.0              4.0   

                                                      \
                                         antecedents   
0  (spread, inventor, nineyearold, wash, way, hands)   
1                               (theories, consider)   
2  (verified, never, theories, conspiracy, things...   
3  (verified, facts, theories, conspiracy, things...   
4  (verified, facts, never, conspiracy, things, p...   

                                                                 
                                         consequents confidence  
0                                             (come)        1.0  
1  (verified, month, tes, facts, conspiracy, neve...        1.0  
2                                            (facts)        1.0  
3                                            (never)        1.0  
4                                         (theories)        1.0

<h3><b>Entropy of all words

In [33]:
from math import log2
def shannon(freq,total):
    
    return(-freq / total * log2(freq / total))

def getWordRank(x,y):
    no_words = x.iloc[:,1:].sum(axis=0)
    tot_words = x.iloc[:,1:].sum().sum()
    word_rank = pd.DataFrame(no_words.apply(lambda p: p/tot_words))
    word_rank['words'] = word_rank.index
    word_rank['count'] = x.iloc[:,1:].sum(axis=0).values
    
    entr_words = no_words.apply(lambda x: shannon(x,no_words.sum()))
    word_rank['entropy'] = entr_words.values
    
    word_rank.sort_values(0,axis = 0, ascending = False, inplace = True)
    word_rank.reset_index(drop=True,inplace=True)
    word_rank.rename({0:'probability'},axis=1,inplace=True)
    word_rank['rank'] = word_rank.index.values
    
    
    no_words = y.iloc[:,1:].sum(axis=0)
    tot_words = y.iloc[:,1:].sum().sum()
    word_rank1 = pd.DataFrame(no_words.apply(lambda p: p/tot_words))
    word_rank1['words'] = word_rank1.index
    word_rank1['count'] = y.iloc[:,1:].sum(axis=0).values
    
    entr_words = no_words.apply(lambda x: shannon(x,no_words.sum()))
    word_rank1['entropy'] = entr_words.values
    
    word_rank1.sort_values(0,axis = 0, ascending = False, inplace = True)
    word_rank1.reset_index(drop=True,inplace=True)
    word_rank1.rename({0:'probability'},axis=1,inplace=True)
    word_rank1['rank'] = word_rank1.index.values
    
    
    col = ['rank','words','count','probability','entropy']
    
    return(word_rank[col],word_rank1[col])

In [34]:
wordRank_withStem, wordRank_withoutStem = getWordRank(df_withStem,df_withoutStem)

In [35]:
#example data
wordRank_exwithStem, wordRank_exwithoutStem = getWordRank(df_exWithStem,df_exWithoutStem)

wordRank_exwithStem.columns = pd.MultiIndex.from_product([['With_Stemming'], wordRank_exwithStem.columns])
wordRank_exwithoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], wordRank_exwithoutStem.columns])

wordRank_exresult = pd.concat([wordRank_exwithStem, wordRank_exwithoutStem], axis = 1)
wordRank_exresult.head(5)

With_Stemming                                      Without_Stemming  \
           rank      words count probability entropy             rank   
0           0.0      death   2.0      0.1250   0.375                0   
1           1.0  bioweapon   1.0      0.0625   0.250                1   
2           2.0    destroy   1.0      0.0625   0.250                2   
3           3.0    economi   1.0      0.0625   0.250                3   
4           4.0     govern   1.0      0.0625   0.250                4   

                                         
        words count probability entropy  
0  bioweapons     1      0.0625    0.25  
1       death     1      0.0625    0.25  
2      deaths     1      0.0625    0.25  
3     destroy     1      0.0625    0.25  
4     economy     1      0.0625    0.25

In [36]:
#origle dataset
wordRank_withStem.columns = pd.MultiIndex.from_product([['With_Stemming'], wordRank_withStem.columns])
wordRank_withoutStem.columns = pd.MultiIndex.from_product([['Without_Stemming'], wordRank_withoutStem.columns])

wordRank_result = pd.concat([wordRank_withStem, wordRank_withoutStem], axis = 1)

In [37]:
wordRank_result.head()

With_Stemming                                      Without_Stemming  \
           rank   words  count probability   entropy             rank   
0           0.0  nation  616.0    0.020930  0.116755                0   
1           1.0    help  544.0    0.018484  0.106423                1   
2           2.0  doctor  534.0    0.018144  0.104952                2   
3           3.0   claim  526.0    0.017872  0.103769                3   
4           4.0    line  523.0    0.017770  0.103324                4   

                                        
     words count probability   entropy  
0   nation   540    0.018348  0.105836  
1  doctors   525    0.017838  0.103621  
2     line   519    0.017634  0.102729  
3  helping   519    0.017634  0.102729  
4    front   518    0.017600  0.102580

<h3><b>Identify all Named entity occurred in all tweets

In [38]:
def getEntityRank(data):
    text = []
    label = []
    disc = []
    def show_ents(doc):
    # Write a function to display basic entity info:
        if doc.ents:
            for ent in doc.ents:
                text.append(ent.text)
                label.append(ent.label_)
                disc.append(str(spacy.explain(ent.label_)))
        return([text,label,disc])
    ner = data.apply(lambda x: show_ents(nlp(x)))
    df_ner = pd.DataFrame([text,label,disc]).transpose()
    df_ner.columns = ['text','ent','discription']

    ner_des = pd.DataFrame(df_ner['ent'].value_counts())
    ner_des['entity'] = ner_des.index
    ner_des.reset_index(drop=True,inplace=True)
    ner_des['discription'] = pd.DataFrame(df_ner['discription'].value_counts()).index
    ner_des.rename({'ent':'count'},axis=1,inplace=True)
    ner_des = ner_des[['entity','discription','count']]

    ner_des['probability'] = ner_des['count'].apply(lambda x: x/ner_des['count'].sum())
    ner_des['rank'] = ner_des.index
    
    return(ner_des)

In [39]:
ner_des = getEntityRank(clean_data)

In [40]:
ner_des

,entity,discription,count,probability,rank
0,DATE,Absolute or relative dates or periods,737,0.213809,0
1,ORG,"Companies, agencies, institutions, etc.",681,0.197563,1
2,GPE,"Countries, cities, states",636,0.184508,2
3,PERSON,"People, including fictional",615,0.178416,3
4,NORP,Nationalities or religious or political groups,381,0.110531,4
5,CARDINAL,Numerals that do not fall under another type,117,0.033943,5
6,TIME,Times smaller than a day,53,0.015376,6
7,MONEY,"Monetary values, including unit",43,0.012475,7
8,ORDINAL,"""first"", ""second"", etc.",38,0.011024,8
9,PRODUCT,"Non-GPE locations, mountain ranges, bodies of ...",33,0.009574,9


<h3><b>Information gain of all possible pair of words from without stemmed data

In [66]:
#[wordRank_withoutStem['words'='line']]

In [67]:
cnt[cnt['words']=='line']['count'].values

array([519])

In [103]:
cnt = wordRank_withoutStem['Without_Stemming']
H_data = cnt['count'].apply(lambda x: shannon(x,cnt['count'].sum())).sum()

def infog(H_x,H_y,cnt_x,cnt_y):
    I_xy = H_data -(cnt_x/cnt_data * H_x + cnt_y/cnt_data * H_y)
    return(I_xy)

entr_words =wordRank_exresult['Without_Stemming']

fin = []
for i in entr_words.iterrows():
    #print(i[1]['entropy'])
    #print('...')
    res = []
    for x in entr_words.iterrows():
        res.append(infog(i[1]['entropy'],x[1]['entropy'],i[1]['count'],x[1]['count']))
    fin.append(res)
    
pd.DataFrame(fin,columns=entr_words['words'].values,index=entr_words['words'].values).iloc[:5,:5]

,bioweapons,death,deaths,destroy,economy
bioweapons,9.762707,9.762707,9.762707,9.762707,9.762707
death,9.762707,9.762707,9.762707,9.762707,9.762707
deaths,9.762707,9.762707,9.762707,9.762707,9.762707
destroy,9.762707,9.762707,9.762707,9.762707,9.762707
economy,9.762707,9.762707,9.762707,9.762707,9.762707


<h3><b> Cosine similarity of each documents of with stemming and without stemming

In [104]:
#example data
a=pd.DataFrame(cosine_similarity(df_exWithoutStem, df_exWithoutStem))
b=pd.DataFrame(cosine_similarity(df_exWithStem, df_exWithStem))
a.columns = pd.MultiIndex.from_product([['With_Stemming'], a.columns])
b.columns = pd.MultiIndex.from_product([['Without_Stemming'], b.columns])

cosine_exResult = pd.concat([a, b], axis = 1)
cosine_exResult

With_Stemming                Without_Stemming               
              0    1         2                0    1         2
0      1.000000  0.0  0.142857         1.000000  0.0  0.125988
1      0.000000  1.0  0.000000         0.000000  1.0  0.000000
2      0.142857  0.0  1.000000         0.125988  0.0  1.000000

In [105]:
docs_similarity_withStem = pd.DataFrame(cosine_similarity(df_withStem, df_withStem))
docs_similarity_withStem.head()

,0,1,2,3,4,5,6,7,8,9,...,3106,3107,3108,3109,3110,3111,3112,3113,3114,3115
0,1.000000,0.087039,0.113961,0.0,0.113961,0.090909,0.083624,0.0,0.090909,0.100504,...,0.0,1.000000,0.150756,0.087039,0.087039,0.0,0.090909,0.087039,0.0,0.123091
1,0.087039,1.000000,0.109109,0.0,0.109109,0.087039,0.080064,0.0,0.087039,0.096225,...,0.0,0.087039,0.144338,0.083333,0.083333,0.0,0.087039,0.083333,0.0,0.117851
2,0.113961,0.109109,1.000000,0.0,0.285714,0.113961,0.209657,0.0,0.113961,0.125988,...,0.0,0.113961,0.188982,0.109109,0.109109,0.0,0.113961,0.218218,0.0,0.154303
3,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,0.113961,0.109109,0.285714,0.0,1.000000,0.113961,0.104828,0.0,0.113961,0.125988,...,0.0,0.113961,0.188982,0.109109,0.109109,0.0,0.113961,0.218218,0.0,0.154303


In [106]:
docs_similarity_withoutStem = pd.DataFrame(cosine_similarity(df_withoutStem, df_withoutStem))
docs_similarity_withoutStem.head()

,0,1,2,3,4,5,6,7,8,9,...,3106,3107,3108,3109,3110,3111,3112,3113,3114,3115
0,1.000000,0.087039,0.113961,0.0,0.113961,0.090909,0.083624,0.0,0.090909,0.100504,...,0.0,1.000000,0.150756,0.087039,0.087039,0.0,0.090909,0.087039,0.0,0.123091
1,0.087039,1.000000,0.109109,0.0,0.109109,0.087039,0.080064,0.0,0.087039,0.096225,...,0.0,0.087039,0.144338,0.083333,0.083333,0.0,0.087039,0.083333,0.0,0.117851
2,0.113961,0.109109,1.000000,0.0,0.285714,0.113961,0.104828,0.0,0.113961,0.125988,...,0.0,0.113961,0.188982,0.109109,0.109109,0.0,0.113961,0.218218,0.0,0.154303
3,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,0.113961,0.109109,0.285714,0.0,1.000000,0.113961,0.104828,0.0,0.113961,0.125988,...,0.0,0.113961,0.188982,0.109109,0.109109,0.0,0.113961,0.218218,0.0,0.154303


<h3><b>Top words from all events using Uncertinity score

In [116]:
#probability of hashtags/event
cnt_tags = []
for i in utags:
    cnt_tags.append(len(df[df['text'].apply(lambda x: i in x)]))

df_tags = pd.DataFrame([utags,cnt_tags],index=['tags','count']).transpose()

total_tags  = sum(cnt_tags) #len(utags)
p_allTags = df_tags['probability'] = df_tags['count'].apply(lambda x: x/total_tags)
p_allTags

0      0.051463
1      0.008643
2      0.000786
3      0.152033
4      0.044196
         ...   
934    0.000196
935    0.000196
936    0.000196
937    0.000196
938    0.000196
Name: count, Length: 939, dtype: float64

In [117]:
#probability of words
total_w = len(df_withoutStem.iloc[:,1:].columns)
p_AllWords = df_withoutStem.iloc[:,1:].sum().apply(lambda x: x/total_w)
p_AllWords

abandon              0.000212
abbey                0.000212
abdiaziz             0.000212
abilene              0.000212
able                 0.000423
                       ...   
zoey                 0.000212
zombieapocalyptic    0.000212
zombies              0.000212
zone                 0.002538
zoom                 0.000423
Length: 4728, dtype: float64

In [118]:
#addition P(tags) + P(words)
p_add=[]
for i in p_allTags:
    p_add.append(p_AllWords.apply(lambda x: x+i))
    
prob_add = pd.DataFrame(p_add)

In [121]:
#Intersection  P(tags I word) = n(words from specific hashtag) / n(total words from all hashtags)

total_wsI = df_withoutStem.iloc[:,1:].sum().sum()
p_TagIword = []
for i in utags:
    df_tagWord = df[df['text'].apply(lambda x: i in x)]
    df_tagWord.reset_index(inplace=True)
    clean_data_tags =df_tagWord['text'].apply(lambda x: p.clean(x))
    df_wsI,df_sI = countVect(clean_data_tags)
   # print(total_wsI)
    p_TagIword.append(df_wsI.iloc[:,1:].sum().apply(lambda x: x/total_wsI))
    

prob_TagIword = pd.DataFrame(p_TagIword)
prob_TagIword.fillna(0,inplace=True)

In [122]:
#union 
comman_colms = np.intersect1d(prob_add.columns,prob_TagIword.columns)
prob_union = prob_add[comman_colms] - prob_TagIword[comman_colms]

col_add = []
col_Intr=[]
for i in prob_add.columns:
    if i not in comman_colms:
        col_add.append(i)
           
for j in prob_TagIword.columns:    
    if j not in comman_colms:
        col_Intr.append(j)
        
uni_prob = prob_union.join(prob_add[col_add]).join(prob_TagIword[col_Intr])
uni_prob = uni_prob[uni_prob.columns.sort_values().values]

In [123]:
#Uncertinity/ conditinal probability
#P(word/tags) = P( tags intersection word) / P(tags)

prob_uncertinity = prob_TagIword.div(p_allTags,axis=0)

prob_uncertinity.index = utags
prob_uncertinity

,about,action,additional,administration,advises,against,aid,amp,an,and,...,sanitizer,images,mapping,momo,omochi,preview,projection,keeps,restarting,tractors
#…,0.001320,0.00066,0.000660,0.000660,0.00066,0.000660,0.00066,0.001320,0.161097,0.164398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Covid_19,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.003931,0.000000,0.102216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#CoronaVirusUpdate,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.043245,0.000000,0.043245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#coronavirus,0.008493,0.00000,0.000447,0.000223,0.00000,0.002011,0.00067,0.011174,0.003576,0.032853,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#COVID,0.006150,0.00000,0.000000,0.000000,0.00000,0.001538,0.00000,0.036903,0.003844,0.019989,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#cities,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.172981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#inequalities…,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.172981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#MayDayStrike…,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Bengal,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
#words ranking of single hashtags
sample = pd.DataFrame(prob_uncertinity.iloc[40,:].sort_values(ascending=False)).head(50)
r = []
for i in range(len(sample)):
    r.append(i)

sample['Rank'] = r
sample.head()

,#ldnont,Rank
poised,0.172981,0
for,0.172981,1
a,0.172981,2
big,0.172981,3
column,0.172981,4


In [126]:
#top ten hashtags
top_tags = df_tags.sort_values(by='probability',ascending=False)['tags'].head(10)

In [127]:
#Top ten words of each hashtags
label=[]
coln=[]
val=[]
for i in range(len(prob_uncertinity)):

    s = pd.DataFrame(prob_uncertinity.iloc[i,:].sort_values(ascending=False).head(10))
    label.append(s.index)
    coln.append(s.columns)
    val.append(s)

top_wordsWithHash = pd.DataFrame(label,index=pd.DataFrame(coln).values).transpose()

In [128]:
top_wordsWithHash[[top_tags]]

,"(#corona,)","(#coronavirus,)","(#Corona,)","(#Coronavirus,)","(#…,)","(#COVID,)","(#COVID19,)","(#CoronaVirus,)","(#Covid_19,)","(#covid,)"
0,the,the,the,the,the,the,to,the,the,to
1,to,to,to,to,to,to,the,a,and,a
2,of,of,of,of,of,of,in,and,has,the
3,a,a,and,in,and,in,of,of,deaths,you
4,in,in,in,and,this,amp,amp,is,from,are
5,and,and,on,on,has,is,that,for,were,of
6,from,from,a,has,reduce,this,deaths,to,revised,for
7,is,is,is,people,up,that,cases,true,actually,and
8,for,for,for,with,with,deaths,this,in,cdc,is
9,on,on,has,remdesivir,spread,from,may,deaths,k,this
